In [1]:
import requests
import json
from genericpath import exists
import os
import pandas as pd

In [17]:
nom="meetings"
path_table=os.walk(f"/workspaces/F1_website/data/raw/{nom}")

df=pd.DataFrame()
for fichiers in path_table:
    for fichier in fichiers[2]:
        with open(fichiers[0]+"/"+fichier) as f:
            data = json.load(f)
            df=pd.concat([df,pd.json_normalize(data)])

display(df)

,meeting_key,meeting_name,meeting_official_name,location,country_key,country_code,country_name,country_flag,circuit_key,circuit_short_name,circuit_type,circuit_image,gmt_offset,date_start,date_end,year
0,1228,Pre-Season Testing,FORMULA 1 ARAMCO PRE-SEASON TESTING 2024,Sakhir,36,BRN,Bahrain,https://media.formula1.com/content/dam/fom-web...,63,Sakhir,Permanent,https://media.formula1.com/content/dam/fom-web...,03:00:00,2024-02-21T07:00:00+00:00,2024-02-23T16:00:00+00:00,2024
1,1229,Bahrain Grand Prix,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Sakhir,36,BRN,Bahrain,https://media.formula1.com/content/dam/fom-web...,63,Sakhir,Permanent,https://media.formula1.com/content/dam/fom-web...,03:00:00,2024-02-29T11:30:00+00:00,2024-03-02T17:00:00+00:00,2024
2,1230,Saudi Arabian Grand Prix,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2024,Jeddah,153,KSA,Saudi Arabia,https://media.formula1.com/content/dam/fom-web...,149,Jeddah,Temporary - Street,https://media.formula1.com/content/dam/fom-web...,03:00:00,2024-03-07T13:30:00+00:00,2024-03-09T19:00:00+00:00,2024
3,1231,Australian Grand Prix,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2024,Melbourne,5,AUS,Australia,https://media.formula1.com/content/dam/fom-web...,10,Melbourne,Temporary - Street,https://media.formula1.com/content/dam/fom-web...,11:00:00,2024-03-22T01:30:00+00:00,2024-03-24T06:00:00+00:00,2024
4,1232,Japanese Grand Prix,FORMULA 1 MSC CRUISES JAPANESE GRAND PRIX 2024,Suzuka,4,JPN,Japan,https://media.formula1.com/content/dam/fom-web...,46,Suzuka,Permanent,https://media.formula1.com/content/dam/fom-web...,09:00:00,2024-04-05T02:30:00+00:00,2024-04-07T07:00:00+00:00,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,1272,Mexico City Grand Prix,FORMULA 1 GRAN PREMIO DE LA CIUDAD DE MÉXICO 2025,Mexico City,8,MEX,Mexico,https://media.formula1.com/content/dam/fom-web...,65,Mexico City,Permanent,https://media.formula1.com/content/dam/fom-web...,-06:00:00,2025-10-24T18:30:00+00:00,2025-10-26T22:00:00+00:00,2025
21,1273,São Paulo Grand Prix,FORMULA 1 MSC CRUISES GRANDE PRÊMIO DE SÃO PAU...,São Paulo,10,BRA,Brazil,https://media.formula1.com/content/dam/fom-web...,14,Interlagos,Permanent,https://media.formula1.com/content/dam/fom-web...,-03:00:00,2025-11-07T14:30:00+00:00,2025-11-09T19:00:00+00:00,2025
22,1274,Las Vegas Grand Prix,FORMULA 1 HEINEKEN LAS VEGAS GRAND PRIX 2025,Las Vegas,19,USA,United States,https://media.formula1.com/content/dam/fom-web...,152,Las Vegas,Temporary - Street,https://media.formula1.com/content/dam/fom-web...,-08:00:00,2025-11-21T00:30:00+00:00,2025-11-23T06:00:00+00:00,2025
23,1275,Qatar Grand Prix,FORMULA 1 QATAR AIRWAYS QATAR GRAND PRIX 2025,Lusail,149,QAT,Qatar,https://media.formula1.com/content/dam/fom-web...,150,Lusail,Permanent,https://media.formula1.com/content/dam/fom-web...,03:00:00,2025-11-28T13:30:00+00:00,2025-11-30T18:00:00+00:00,2025


In [15]:
nom="drivers"
with open(f"/workspaces/F1_website/data/raw/{nom}/{nom}.json") as f:
    df=pd.json_normalize(json.load(f))

display(df[["meeting_key","team_name","team_colour"]].drop_duplicates())

,meeting_key,team_name,team_colour
0,1140,Red Bull Racing,3671C6
1,1140,Williams,37BEDD
2,1140,McLaren,F58020
3,1140,Alpine,2293D1
5,1140,Aston Martin,358C75
...,...,...,...
7083,1277,Mercedes,00D7B6
7084,1277,Aston Martin,229971
7085,1277,Ferrari,ED1131
7087,1277,Williams,1868DB
